# Food recognition (drafting a solution)
input image including some food -> output label (name of food)

Resources:
- preprocessing with image augmentation to reduce overfitting + useful visualization of results
    - https://www.learndatasci.com/tutorials/convolutional-neural-networks-image-classification/#ImplementationofaCNNinKeras
- vgg16 with food101
    - https://www.kaggle.com/code/kmader/food-train-vgg16-from-scratch/notebook
    - https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/
- building an image data pipeline
    - https://cs230.stanford.edu/blog/datapipeline/#building-an-image-data-pipeline
    - https://www.youtube.com/watch?v=VFEOskzhhbc
- look into transfer learning with vgg-16, to get better results with less data 
    - https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4
    - https://medium.com/@1297rohit/transfer-learning-from-scratch-using-keras-339834b153b9
- look into allowing multi-class prediction / multiple objects in one image
    - https://pyimagesearch.com/2020/10/12/multi-class-object-detection-and-bounding-box-regression-with-keras-tensorflow-and-deep-learning/

## Data Preparation

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from keras_preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from IPython.display import clear_output
import os
from glob import glob
import pandas as pd

### Load food dataset:

(Hub) food-recognition-2022: https://www.kaggle.com/code/sainikhileshreddy/how-to-use-the-dataset/notebook
- multiple object classifications, bounding boxes

(Hub) food-101-dataset-train: https://app.activeloop.ai/activeloop/food-101-dataset-train
- single object classifications

food-101 from file structure usage
- http://blog.stratospark.com/deep-learning-applied-food-classification-deep-learning-keras.html
- https://www.kaggle.com/code/themlphdstudent/food-classification-using-inceptionv3/notebook


In [ ]:
#ds_train_original = hub.load('hub://activeloop/food-101-dataset-train')
#ds_train_original, info = tfds.load('food101', split='train', with_info=True)

# Download raw food101 dataset to file structure.
#   I attempted first to use Hub datasets, however after banging my head against
#   a wall finally gave up because they don't appear to be integrated very well
#   with tensorflow. Or at least, I couldn't figure it out, and kept running into
#   more and more problems the deeper I went. 
#   Second I tried to use tfds.load to load food101 straight as a tf Dataset. 
#   But I couldn't figure out how to get the class label names, and in general
#   am confounded with the enigma of the tf Dataset structure when I'm not the
#   one that creates it, so figured by this point I should just download the 
#   raw files because I know how to deal with a file structure, and I know how 
#   to later add more custom data to said file structure. 
#   (ie, making a data folder for bananas, and adding my own data.)
#   So here we are:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar xzvf food-101.tar.gz
clear_output()

class_names & num_classes

In [ ]:
# save class names into list class_names for later
classes_file = open("food-101/meta/classes.txt", "r")
# read lines into a list of class names, ignoring empty lines
class_names = [line.strip() for line in classes_file.readlines() if line.strip()]
classes_file.close()
# save num_classes
num_classes = len(class_names)
print("number of classes: ",num_classes)

number of classes:  101


Prepare data into a pd dataframe before creating the tf Dataset

In [ ]:
# grab list of all image filepaths
all_images = glob(os.path.join('food-101', 'images', '*', '*'))
# save the image count for later
image_count = len(all_images)
# create the df with path column
df = pd.DataFrame(dict(path = all_images))
# add class_name column
df['class_name'] = df['path'].map(lambda x: x.split('/')[-2])
# add class column
# (class is the one-hot-encoded version of the class_name, ordered the same as the class_names list)
df['class'] = df['class_name'].map(lambda x: to_categorical(class_names.index(x), num_classes=num_classes))
# print image count
print(image_count)
# show sample of df
df.sample(3)

101000


,path,class_name,class
10645,food-101/images/strawberry_shortcake/1722042.jpg,strawberry_shortcake,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
27165,food-101/images/carrot_cake/2425189.jpg,carrot_cake,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
78205,food-101/images/waffles/3014777.jpg,waffles,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Train/test split

In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, 
                                   test_size = 0.2, 
                                   random_state = 2022,
                                   stratify = df['class_name'])
print('train', train_df.shape[0], 'validation', valid_df.shape[0])
train_size = train_df.shape[0]
valid_size = valid_df.shape[0]

train 80800 validation 20200


Create train and validation tf Datasets using info in the dataframes

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_df['path'].values, np.stack(train_df['class'].values,0)))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_df['path'].values, np.stack(valid_df['class'].values,0)))

In [ ]:
# Given tensor tuple (img_path, label), return (img, label)
@tf.function
def parse_image(img_path, label):
    img = tf.io.read_file(img_path) # load image file data as string
    img = tf.image.decode_jpeg(img, channels=3) # encode as jpg, 3 channels (RGB)
    return img, label

## Implementing with VGG-16

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

# given tensor tuple (img, label), process the image to be ready for vgg16 input
# return (img, label)
@tf.function
def process_image(img, label):
  # Resize images
  img = tf.image.resize(img, [224, 224]) # vgg16 expects 224x224 images
  # rescale to float values in 0-1 (Actually I think this is not done for vgg16 input, commenting this out)
  #img = img/255 
  # preprocess the input to fit vgg-16 inputs using given preprocessing function
  img = preprocess_input(img)
  # return as image, label tuple
  return img, label

In [ ]:
train_ds = train_ds.shuffle(train_size).map(parse_image).map(process_image).batch(32)
valid_ds = valid_ds.shuffle(valid_size).map(parse_image).map(process_image).batch(32)

Load the base pretrained vgg-16 model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

# load the VGG-16 model from keras
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3)) # include_top=False to remove the classification layers for the original dataset
base_model.trainable = False # Not trainable weights; we don't want to modify the pretrained ImageNet weights

58900480/58889256 [==============================] - 1s 0us/step


Add the last layers to train for our specific problem

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    base_model, # VGG-16 base model
    layers.Flatten(), # flatten layer
    layers.Dense(4096, activation='relu'), # dense layer 1
    layers.Dense(1072, activation='relu'), # dense layer 2
    layers.Dropout(0.2), # Dropout layer to combat overfitting
    layers.Dense(num_classes, activation='softmax') # prediction layer
])

Compile & train

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

model.compile(
    optimizer='adam', # adam defaults to learning rate=0.001
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# ModelCheckpoint callback - save best weights after each epoch(?)
checkpoint = ModelCheckpoint(filepath='model_v1.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# Early Stopping callback - stops early if no improvement is noted, restoring best weights
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=7,  restore_best_weights=True)

history = model.fit(train_ds, validation_data=valid_ds, epochs=25, callbacks=[es, checkpoint]) #epochs=50,

Epoch 1/25
2525/2525 [==============================] - ETA: 0s - loss: 5.0317 - accuracy: 0.0599
Epoch 1: val_loss improved from inf to 3.86480, saving model to model_v1.weights.best.hdf5
2525/2525 [==============================] - 1037s 405ms/step - loss: 5.0317 - accuracy: 0.0599 - val_loss: 3.8648 - val_accuracy: 0.1247
Epoch 2/25
2525/2525 [==============================] - ETA: 0s - loss: 3.6073 - accuracy: 0.1470
Epoch 2: val_loss improved from 3.86480 to 3.13467, saving model to model_v1.weights.best.hdf5
2525/2525 [==============================] - 1020s 404ms/step - loss: 3.6073 - accuracy: 0.1470 - val_loss: 3.1347 - val_accuracy: 0.2214
Epoch 3/25
2525/2525 [==============================] - ETA: 0s - loss: 3.2096 - accuracy: 0.2149
Epoch 3: val_loss improved from 3.13467 to 3.05641, saving model to model_v1.weights.best.hdf5
2525/2525 [==============================] - 1020s 404ms/step - loss: 3.2096 - accuracy: 0.2149 - val_loss: 3.0564 - val_accuracy: 0.2650
Epoch 4/25


In [ ]:
""" here is the training log before it disconnected at epoch 23
Epoch 1/25
2525/2525 [==============================] - ETA: 0s - loss: 5.0317 - accuracy: 0.0599
Epoch 1: val_loss improved from inf to 3.86480, saving model to model_v1.weights.best.hdf5
2525/2525 [==============================] - 1037s 405ms/step - loss: 5.0317 - accuracy: 0.0599 - val_loss: 3.8648 - val_accuracy: 0.1247
Epoch 2/25
2525/2525 [==============================] - ETA: 0s - loss: 3.6073 - accuracy: 0.1470
Epoch 2: val_loss improved from 3.86480 to 3.13467, saving model to model_v1.weights.best.hdf5
2525/2525 [==============================] - 1020s 404ms/step - loss: 3.6073 - accuracy: 0.1470 - val_loss: 3.1347 - val_accuracy: 0.2214
Epoch 3/25
2525/2525 [==============================] - ETA: 0s - loss: 3.2096 - accuracy: 0.2149
Epoch 3: val_loss improved from 3.13467 to 3.05641, saving model to model_v1.weights.best.hdf5
2525/2525 [==============================] - 1020s 404ms/step - loss: 3.2096 - accuracy: 0.2149 - val_loss: 3.0564 - val_accuracy: 0.2650
Epoch 4/25
2525/2525 [==============================] - ETA: 0s - loss: 2.9507 - accuracy: 0.2671
Epoch 4: val_loss improved from 3.05641 to 2.92235, saving model to model_v1.weights.best.hdf5
2525/2525 [==============================] - 1019s 404ms/step - loss: 2.9507 - accuracy: 0.2671 - val_loss: 2.9223 - val_accuracy: 0.2786
Epoch 5/25
2525/2525 [==============================] - ETA: 0s - loss: 2.7670 - accuracy: 0.3023
Epoch 5: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1013s 401ms/step - loss: 2.7670 - accuracy: 0.3023 - val_loss: 2.9691 - val_accuracy: 0.3067
Epoch 6/25
2525/2525 [==============================] - ETA: 0s - loss: 2.6367 - accuracy: 0.3326
Epoch 6: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1014s 401ms/step - loss: 2.6367 - accuracy: 0.3326 - val_loss: 2.9838 - val_accuracy: 0.3192
Epoch 7/25
2525/2525 [==============================] - ETA: 0s - loss: 2.5114 - accuracy: 0.3584
Epoch 7: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1012s 401ms/step - loss: 2.5114 - accuracy: 0.3584 - val_loss: 2.9428 - val_accuracy: 0.3259
Epoch 8/25
2525/2525 [==============================] - ETA: 0s - loss: 2.3995 - accuracy: 0.3827
Epoch 8: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1012s 401ms/step - loss: 2.3995 - accuracy: 0.3827 - val_loss: 3.1809 - val_accuracy: 0.3332
Epoch 9/25
2525/2525 [==============================] - ETA: 0s - loss: 2.3301 - accuracy: 0.3983
Epoch 9: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1015s 402ms/step - loss: 2.3301 - accuracy: 0.3983 - val_loss: 3.1236 - val_accuracy: 0.3407
Epoch 10/25
2525/2525 [==============================] - ETA: 0s - loss: 2.2616 - accuracy: 0.4149
Epoch 10: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1011s 401ms/step - loss: 2.2616 - accuracy: 0.4149 - val_loss: 3.2407 - val_accuracy: 0.3391
Epoch 11/25
2525/2525 [==============================] - ETA: 0s - loss: 2.2022 - accuracy: 0.4298
Epoch 11: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1011s 400ms/step - loss: 2.2022 - accuracy: 0.4298 - val_loss: 3.3298 - val_accuracy: 0.3475
Epoch 12/25
2525/2525 [==============================] - ETA: 0s - loss: 2.1185 - accuracy: 0.4438
Epoch 12: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1011s 400ms/step - loss: 2.1185 - accuracy: 0.4438 - val_loss: 3.3182 - val_accuracy: 0.3305
Epoch 13/25
2525/2525 [==============================] - ETA: 0s - loss: 2.0906 - accuracy: 0.4503
Epoch 13: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1015s 402ms/step - loss: 2.0906 - accuracy: 0.4503 - val_loss: 3.5828 - val_accuracy: 0.3503
Epoch 14/25
2525/2525 [==============================] - ETA: 0s - loss: 2.0158 - accuracy: 0.4662
Epoch 14: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1013s 401ms/step - loss: 2.0158 - accuracy: 0.4662 - val_loss: 3.5949 - val_accuracy: 0.3524
Epoch 15/25
2525/2525 [==============================] - ETA: 0s - loss: 1.9665 - accuracy: 0.4767
Epoch 15: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1011s 400ms/step - loss: 1.9665 - accuracy: 0.4767 - val_loss: 3.4672 - val_accuracy: 0.3456
Epoch 16/25
2525/2525 [==============================] - ETA: 0s - loss: 1.9766 - accuracy: 0.4842
Epoch 16: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1012s 401ms/step - loss: 1.9766 - accuracy: 0.4842 - val_loss: 4.4349 - val_accuracy: 0.3546
Epoch 17/25
2525/2525 [==============================] - ETA: 0s - loss: 1.9322 - accuracy: 0.4917
Epoch 17: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1005s 398ms/step - loss: 1.9322 - accuracy: 0.4917 - val_loss: 4.2226 - val_accuracy: 0.3556
Epoch 18/25
2525/2525 [==============================] - ETA: 0s - loss: 1.9084 - accuracy: 0.4956
Epoch 18: val_loss did not improve from 2.92235
2525/2525 [==============================] - 998s 395ms/step - loss: 1.9084 - accuracy: 0.4956 - val_loss: 4.0911 - val_accuracy: 0.3544
Epoch 19/25
2525/2525 [==============================] - ETA: 0s - loss: 1.8745 - accuracy: 0.5068
Epoch 19: val_loss did not improve from 2.92235
2525/2525 [==============================] - 999s 396ms/step - loss: 1.8745 - accuracy: 0.5068 - val_loss: 4.5510 - val_accuracy: 0.3573
Epoch 20/25
2525/2525 [==============================] - ETA: 0s - loss: 1.8818 - accuracy: 0.5088
Epoch 20: val_loss did not improve from 2.92235
2525/2525 [==============================] - 999s 396ms/step - loss: 1.8818 - accuracy: 0.5088 - val_loss: 3.9220 - val_accuracy: 0.3408
Epoch 21/25
2525/2525 [==============================] - ETA: 0s - loss: 1.8526 - accuracy: 0.5155
Epoch 21: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1001s 396ms/step - loss: 1.8526 - accuracy: 0.5155 - val_loss: 4.5267 - val_accuracy: 0.3433
Epoch 22/25
2525/2525 [==============================] - ETA: 0s - loss: 1.8511 - accuracy: 0.5192
Epoch 22: val_loss did not improve from 2.92235
2525/2525 [==============================] - 1001s 396ms/step - loss: 1.8511 - accuracy: 0.5192 - val_loss: 4.5260 - val_accuracy: 0.3517
Epoch 23/25
 778/2525 [========>.....................] - ETA: 9:26 - loss: 1.7616 - accuracy: 0.5303
 """

## Learning how to use tf dataset (temporary)

In [ ]:
# ds_train_X = ds_val_original.images.numpy(aslist=True)
# ds_train_y = ds_val_original.categories.numpy(aslist=True)

# plt.imshow(ds_train_X[9])
# print(ds_train_y[9])
# for i in ds_train_y[9]:
#     print(categories[i])

# ds_val_original[0].categories.numpy().size